# Custom priors

The prior probability is a critical element of Bayes theorem.
However, to keep `uravu` straightforward to use, by default, a broad uniform prior probability is assigned to the `Relationship` object. 

Of course this may be ignored and custom priors may be used (*and sometimes it may be necessary that this is done*).
This tutorial will show **how** custom priors may be used with `uravu`. 

Let's start as always be producing some synthetic data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(2)

In [ ]:
x = np.linspace(10, 50, 20)
y = .3 * x ** 2 - 1.4 * x + .2
y += y * np.random.randn(20) * 0.05
dy = 3 * x

In [ ]:
plt.errorbar(x, y, dy, marker='o', ls='')
plt.show()

The model for this data is a two degree polynomial, below is a function that defines this. 
The `Relationship` object is also created.

In [ ]:
def two_degree(x, a, b, c):
    return c * x ** 2 + b * x + a

In [ ]:
from uravu.relationship import Relationship

In [ ]:
modeller = Relationship(two_degree, x, y, dy)
modeller.max_likelihood()

The max likelihood (which makes no consideration of the prior) is found, 

In [ ]:
print(modeller.variable_medians)

The default prior probabilities for these variables with `uravu` are uniform in the range $[x - 10x, x + 10x)$, where $x$ is the current value of the variable.

However, if you wanted the prior probability to be a normal distribution, centred on the current value of the varible with a width of 1, it would be necessary to create a custom prior function. 
This function is shown below. 

In [ ]:
from scipy.stats import norm

def custom_prior():
    priors = []
    for var in modeller.variable_medians:
        priors.append(norm(loc=var, scale=1))
    return priors

Note that the function returns a list of 'frozen' `scipy` RV objects that describe the shape of the priors. 

To make use of these priors, they must be passed to the `mcmc` or `nested_sampling` functions as the `prior_function` keyword argument. 

In [ ]:
modeller.mcmc(prior_function=custom_prior)

In [ ]:
modeller.nested_sampling(prior_function=custom_prior)

In [ ]:
print(modeller.ln_evidence)

Any `scipy` [statistical function](https://docs.scipy.org/doc/scipy/reference/stats.html) that has a `logpdf` class method may be used in the definition of priors. 